In [1]:
import google.protobuf
!pip install protobuf==3.20.1


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW



2023-10-09 08:16:14.898116: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
! pip install transformers datasets

# 東北大学の日本語用BERT使用に必要なパッケージをインストール
! pip install fugashi ipadic


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
!nvidia-smi

Mon Oct  9 08:17:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   45C    P8    19W / 350W |    767MiB / 24234MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [6]:
checkpoint = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
# 前処理関数: tokenize_function
# import numpy as np
# from datasets import Dataset
# def tokenize_function(batch):
#     tokenized_batch = tokenizer(batch['発話者'], batch['応答者'], truncation=True, padding='max_length')
#     #tokenized_batch['labels'] = batch['対話行為list']
#     tokenized_batch['labels'] = np.argmax(batch['対話行為list'], axis=-1)
#     return tokenized_batch

# # Transformers用のデータセット形式に変換
# # pandas.DataFrame -> datasets.Dataset
# target_columns = ['発話者', "応答者","対話行為list"]

# train_dataset = Dataset.from_pandas(train[target_columns])
# val_dataset = Dataset.from_pandas(val[target_columns])
# test_dataset = Dataset.from_pandas(test[target_columns])
# batch_size = 10
# # 前処理（tokenize_function） を適用
# train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
# val_tokenized_dataset = val_dataset.map(tokenize_function, batched=True)
# test_tokenized_dataset = test_dataset.map(tokenize_function, batched=True)

NameError: name 'train' is not defined

In [8]:
model = torch.load('dialogueact_weight.pth')

In [9]:
dialogue_labels = ["自己開示", "質問(YesNo)", '質問(What)', '応答(YesNo)', '応答(平叙)', 'あいづち', 'フィラー', '確認', '要求']

In [10]:
import numpy as np
def np_softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

#def analyze_emotion(text, show_fig=False, ret_prob=False):
def analyze_emotion(text1, text2):
    # 推論モードを有効か
    model.eval()

    # 入力データ変換 + 推論
    tokens = tokenizer(text1, text2, truncation=True, return_tensors="pt")
    tokens.to(model.device)
    preds = model(**tokens)
    #print(preds)
    prob = np_softmax(preds.logits.cpu().detach().numpy()[0])
    out_dict = {n: p for n, p in zip(dialogue_labels, prob)}
    
    max_key = max(out_dict, key=out_dict.get)
    #print(max_key, out_dict[max_key])

    

    # 棒グラフを描画
    #if show_fig:
        #plt.figure(figsize=(8, 3))
        #df = pd.DataFrame(out_dict.items(), columns=['name', 'prob'])
        #sns.barplot(x='name', y='prob', data=df)
        #plt.title('入力文 : ' + text, fontsize=15)

    #if ret_prob:
        #return out_dict
    #return out_dict
    return max_key

# 動作確認
analyze_emotion("そうなの！チラッと見たら全然移りが違うの。真剣な表情くっきり！","Aも買っちゃえば？高いかもしれないけど")

'質問(YesNo)'

In [13]:
df = pd.read_csv("empathetic_bunrui.csv")
df.head()

,ID,A,B,感情
0,1_1,寝ているときに首がもぞもぞしたので、起きて見てみたらムカデがいたの！,きゃーっ！それはびっくりだね。,おどろく
1,1_2,妹が先日出産しまして。赤ちゃんに会いに病院に行ってきました。,おめでとうございます。男の子、女の子？,わくわくする
2,1_3,立っていたら、いきなり膝カックンされた！,あはは。油断しちゃったのね。,怒る
3,1_4,聞いてください、うちの息子がやりました。,何かしでかしたの？,誇りに思う
4,1_5,あーがっくり。私の親友が、転校してしまいました。,それは、がっくりだね。,悲しい


In [15]:
hatuwabun_A = df["A"].to_list()
hatuwabun_B = df["B"].to_list()

In [23]:
act = []
for i in range(len(hatuwabun_A)):
    result = analyze_emotion(hatuwabun_A[i],hatuwabun_B[i])
    #print(result,hatuwabun_B[i])
    act.append(result)

In [24]:
df["対話行為"] = act

In [27]:
df.tail()

,ID,A,B,感情,対話行為
19995,1_19996,シャッターが下りてる商店街を歩くと、なんだか切なくなる,今は商店街ってよりもスーパーだもんね,感傷的になる,自己開示
19996,1_19997,今担当してもらってる美容師さん、本当に上手なんだ,理想の髪型にカットしてもらえるの？,信頼する,質問(YesNo)
19997,1_19998,サークルで学祭の責任者をやったんだけど、力不足でまとめあげられなかったんだよね,そうだったんだ、残念だったねぇ,恥じる,自己開示
19998,1_19999,来年のオリンピックどうなるのかな？,不安だよね,懸念する,応答(平叙)
19999,1_20000,日本のため、国民のためって働いてくれている誠実な政治家っているのかね？,いないに一票,誠実な気持ち,応答(平叙)


In [35]:
print(len(df[df["感情"] == "おどろく"]))
print(len(df[df["感情"] == "怒る"]))
print(len(df[df["感情"] == "悲しい"]))
print(len(df[df["感情"] == "楽しい"]))
print(len(df[df["感情"] == "期待する"]))
print(len(df[df["感情"] == "信頼する"]))
print(len(df[df["感情"] == "嫌悪感を抱く"]))
print(len(df[df["感情"] == "恐ろしい"]))

625
625
625
625
625
625
625
625


In [38]:
pair = pd.DataFrame()
pair = df[df["感情"] == "おどろく"]
pair = pd.concat([pair, df[df["感情"] == "怒る"]], axis = 0)
pair = pd.concat([pair, df[df["感情"] == "悲しい"]], axis = 0)
pair = pd.concat([pair, df[df["感情"] == "楽しい"]], axis = 0)
pair = pd.concat([pair, df[df["感情"] == "期待する"]], axis = 0)
pair = pd.concat([pair, df[df["感情"] == "信頼する"]], axis = 0)
pair = pd.concat([pair, df[df["感情"] == "嫌悪感を抱く"]], axis = 0)
pair = pd.concat([pair, df[df["感情"] == "恐ろしい"]], axis = 0)

In [42]:
pair.to_csv("dialogueact_huyo.csv", index = False)

In [41]:
pair

,ID,A,B,感情,対話行為
0,1_1,寝ているときに首がもぞもぞしたので、起きて見てみたらムカデがいたの！,きゃーっ！それはびっくりだね。,おどろく,応答(平叙)
32,1_33,そういえば聞いた？あの子プロポーズされたらしいよ。,ええっ、そんな気全然なかったよね。,おどろく,自己開示
64,1_65,昨日、私の誕生日だったんです。娘がケーキを買ってきてくれました。,おめでとうございます。どんなケーキだったんですか。,おどろく,自己開示
96,1_97,先月の私の誕生日にさ、営業部のみんながケーキ買って祝ってくれたの！,へぇ！ いい職場だね。,おどろく,応答(平叙)
128,1_129,ベランダにセミが落ちてたんだよね、お腹を上にしてるから死んでると思ったんだけど。,そんな、季節だね。,おどろく,自己開示
...,...,...,...,...,...
19849,1_19850,ゲームセンターにVRでお化け屋敷の体験ができるコーナーがあってやってみたんだ。,VRだとかなりリアルそうだね。,恐ろしい,自己開示
19881,1_19882,この間の豪雨でこっから50メートルぐらいのところが浸水したんだよね,近いところで浸水しちゃったんだね,恐ろしい,自己開示
19913,1_19914,友達に誘われてケーキバイキングに行ってしまった,美味しかった？,恐ろしい,質問(YesNo)
19945,1_19946,昔話って結構恐ろしい内容多いよね？,例えば？,恐ろしい,質問(YesNo)
